In [ ]:
""" 
import pickle
import pandas as pd

#Obtener lista de usuarios de base de datos original de twisty: 
usuarios = pd.read_json("TwiSty-ES.json",orient="index")["user_id"].tolist()


#Guardar usuarios en un pickle: 
with open("usuarios.pkl","wb") as file:
    pickle.dump(usuarios,file)
    
#---------------------------------------------------------------------------------------------------
#Crear lista de usuarios empty donde ir introduciendo los usuarios hechos: 
usuarios_done = []
with open("usuarios_done.pkl","wb") as file:
    pickle.dump(usuarios_done,file)
    

with open("usuarios.pkl","rb") as f:
    usuarios = pickle.load(f)

# Crear diccionario empty con los ID de usuarios como claves, donde ir guardando sus tweets recogidos:
tweets = {}
for i in usuarios:
    tweets[i] = None
    
with open('tweets.json',"w") as file:
    json.dump(tweets, file)  
    
"""

In [20]:
import tweepy as tw 
import json
import pandas as pd
import pickle

#Obtener claves para conectarse a la api de Twitter:
with open("api_keys.txt","r") as file:      #Las claves deben estar en el archivo .txt
    for line in file:
        exec(line)
    claves = tw.OAuthHandler(consumer_key, consumer_secret)
    claves.set_access_token(access_token, access_token_secret)
    api = tw.API(claves, wait_on_rate_limit=True)

def get_tweets(user,N):
    """ Obtener -al menos- N Tweets de un usuario"""
    try: # Si el usuario no existe o es privado:
        tweets = api.user_timeline(user_id=user, count=200, include_rts=False, tweet_mode='extended')
        corpus = [tweet.full_text for tweet in tweets]
        oldest = tweets[-1].id
    except:
        corpus = []
    if len(corpus) != 0: 
        try: # Si no se pueden obtener mas tweets: 
            while len(corpus) < N:
                tweets_news = api.user_timeline(user_id=user, count=200, include_rts=False, tweet_mode='extended', max_id = oldest -1 )
                new_corpus = [tweet.full_text for tweet in tweets_news]
                oldest = tweets_news[-1].id
                corpus.extend(new_corpus) 
        except:
            pass
        
    return corpus 

def get_data(N):

    #Abrir listas que contienen los usuarios buscados y por buscar y diccionario donde guardar los tweets
    with open("usuarios.pkl","rb") as f1, open ("usuarios_done.pkl","rb") as f2, open("tweets.json") as f3:
        usuarios = pickle.load(f1)
        usuarios_done=pickle.load(f2)
        tweets=json.load(f3)

    contador = 0
        
    for user in [user for user in usuarios if user not in usuarios_done]:
        contador +=1
        corpus_user = get_tweets(user,2000)             # <--------------- N tweets
        tweets[user] = corpus_user
        total = len(corpus_user)
        print(f"Del usuario {contador} de {N} se han recogido {total} Tweets")
        usuarios_done.append(user)
        
        if contador ==N:
            break
            
            
    #Guardar cambios:
    with open("usuarios_done.pkl","wb") as f1, open("tweets.json","w") as f2:
        pickle.dump(usuarios_done,f1)
        json.dump(tweets,f2)
        
    

In [21]:
get_data(50)

Del usuario 1 de 50 se han recogido 1533 Tweets
Del usuario 2 de 50 se han recogido 1517 Tweets
Del usuario 3 de 50 se han recogido 0 Tweets
Del usuario 4 de 50 se han recogido 2132 Tweets
Del usuario 5 de 50 se han recogido 853 Tweets
Del usuario 6 de 50 se han recogido 0 Tweets
Del usuario 7 de 50 se han recogido 897 Tweets
Del usuario 8 de 50 se han recogido 2058 Tweets
Del usuario 9 de 50 se han recogido 0 Tweets
Del usuario 10 de 50 se han recogido 690 Tweets
Del usuario 11 de 50 se han recogido 2014 Tweets
Del usuario 12 de 50 se han recogido 2076 Tweets


2 horas y 45m para mil 

In [1]:

# Total de usuarios: 10772
import pickle
with open("tweets.json", "r") as f1, open("usuarios_done.pkl","rb") as f2:
    tweets = json.load(f1)
    usuarios_done = pickle.load(f2)
    f1.close()
    f2.close()
    
total = []
for i in list(tweets):
    try:
        n = len(tweets[i])
    except:
        n = 0
    total.append(n)

print(f"Total de Tweets recopilados: {sum(total)}")
print(f"Total de usuarios recopilados -No Empty- : {len([x for x in total if x!=0])}")   
print(f"Total de usuarios intentados: {len(usuarios_done)}")


Total de Tweets recopilados: 12995504
Total de usuarios recopilados -No Empty- : 8900
Total de usuarios intentados: 10772
